# Equations, Equilibrium, and Simulation

In [2]:
import numpy as np
import scipy as sp
from scipy.optimize import minimize
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
from matplotlib.lines import Line2D
from scipy.special import lambertw
import pickle
import copy

### Parameters

In [3]:
def paraReset(employees = 50):
    para = {"employees": employees, "r_d": 1./60, #Parameters for the firm size and rule decay rate (rd= 1/Td)
            "k_C": 24, "k_P": 72, #Parameters for the efficacy of work towards creation and pruning
            "A": 1, "alpha" : 0.8, "beta": 0.5, #Parameters for the utility function
            "d": 0.5, "c" : 0.1, #Parameters for the general allocation scheme. This is not the model written in themain text.
            "dtilde" : 0.5, "gamma_c" : 0.5, "gamma_p" : 0.5, "a" : 0.05, # Parameters for the behavioral allocation scheme
            "init_R_u" : 1, "init_R_o" : 0, "last_init_Rs" : [20, 10], #Parameters for the initial rule values (for simulation)
            "discount_rate" : 0.02, #Parameter for discounted optimization
            "b" : 0, #Partial enforcement parameter
            "dt" : 0.01} #Data used in the implementation of this code
    # Additional parameters based on number of employees
    para['w_max'] = int(8*(5/7)*30*para['employees'])
    para['k_A'] = 0.3*para['employees']
    return para

In [4]:
def paraUpdateEmployees(para, employees = 50):
    para['employees'] = employees
    para['w_max'] = int(8*(5/7)*30*para['employees'])
    para['k_A'] = 0.3*para['employees']
    return para

In [5]:
def valid_params(para, general = True, optimize = False):
    if general:
        if para['d'] + para['c'] > 1:
            if not optimize: # We do this because the optimization algorithm
                raise ValueError("Please ensure that d + c <= 1")
            return False
        else:
            return True
    else:
        if 0 > para['dtilde'] or para['dtilde'] > 1 or para['gamma_c'] < 0 or para['gamma_c'] > 1 or para['gamma_p'] < 0 or para['gamma_p'] > 1 or para['b'] < 0:
            #raise ValueError("Please ensure that the values dtilde, gamma_c, and gamma_p are between 0 and 1 and that b is non-negative.")
            return False
        else:
            return True

### Equations

In [6]:
def general_work_eqns(rules, para):
    """
    The work allocations under the general allocation scheme.
    """
    Ru, Ro = rules
    if Ru+Ro == 0:
        raise ValueError("There must be a non-zero number of rules.")
    if Ru < 0 or Ro < 0:
        raise ValueError(f"There must be a posiive number of number of rules: Ru = {Ru}, Ro = {Ro}")
    
    wA = para['k_A']*(Ru+Ro)
    wC = round((para['w_max'] - wA)*para['c'], 8) #Round to eight decimal places to avoid numerical errors
    wD = round((para['w_max'] - wA)*para['d'], 8)
    wP = round((para['w_max'] - wA)*(1-para['c']-para['d']), 8)

    return wA, wD, wC, wP

In [7]:
def f(w_A, para): 
    """
    The partial administration function representing proportion of rules followed.
    """
    return np.exp(-para['b']*(w_A/para['w_max']))

In [8]:
def behavioral_work_eqns(rules, para, return_tildes = False, return_both_wAs = False):
    """
    The work allocations under the behavioral allocation scheme.
    """
    Ru, Ro = rules
    if Ru+Ro == 0:
        raise ValueError("There must be a non-zero number of rules.")

    wA_star = para['k_A']*(Ru + Ro)
    wA = round(wA_star*f(wA_star, para), 8)

    den1 = para['dtilde']
    den2 = (np.exp(-para['a']*Ru*f(wA_star, para)))**(-np.log(para['gamma_c']))
    den3 = np.power(wA, -np.log(para['gamma_p']))/np.power(para['w_max'],-np.log(para['gamma_p']))
    denom = den1 + den2 + den3
    d_temp = para['dtilde']/denom
    c_temp = ((np.exp(-para['a']*Ru*f(wA_star, para)))**(-np.log(para['gamma_c'])))/denom
    wC = np.round((para['w_max']-wA)*c_temp, 8)
    wD = np.round((para['w_max']-wA)*d_temp, 8)
    wP = np.round((para['w_max']-wA)*(1-d_temp-c_temp), 8)

    output = [wA, wD, wC, wP]
    
    if return_tildes:
        output.append(d_temp)
        output.append(c_temp)
        output.append(1-d_temp-c_temp)
    if return_both_wAs:
        output.append(wA_star)
        
    return output

In [9]:
def system_eqns(rules, wC, wP, para):
    """
    Differential equations for the model.
    """
    
    Ru, Ro = rules
    dRudt = wC/para['k_C'] - Ru*para['r_d']
    dRodt = Ru*para['r_d'] - (Ro/(Ru+Ro))*wP/para['k_P']
    
    return [dRudt, dRodt]

### Equilibrium

In [10]:
def equilGeneralAnalytic(pair, para, sign = 1, invalidNan = False, returnStab = False, return_rules = False):
    """
    Utility function for the stable equilibrium point. 
    sign is a parameter that allows us to change the sign of the output. This allows us to use SciPy's Minimize function.
    Solved by Mathematica.
    """
    
    prop_d, prop_c = pair
    prop_p = 1 - prop_d - prop_c
    denominator1 = para['k_C']*(prop_c*para['k_A']*prop_p - prop_c*para['k_P']*para['r_d'] + para['k_C']*prop_p*para['r_d'])
    denominator2 = prop_c*para['k_A']*para['k_C']*prop_p - prop_c*para['k_C']*para['k_P']*para['r_d'] + para['r_d']*prop_p*(para['k_C']**2)
    
    if prop_c + prop_d > 1:
            if invalidNan:
                return np.nan
            else:
                return 0 
     
    if denominator1 != 0 and denominator2 != 0:
        R_u = (prop_c*para['w_max']*(para['k_C']*prop_p - prop_c*para['k_P']))/denominator1
        R_o = (para['k_P']*para['w_max']*(prop_c**2))/denominator2
        if return_rules:
            return (R_u, R_o)

    else:
        if invalidNan:
            return np.nan
        else:
            return 0  
    
    if R_u < 0 or R_o < 0:
        return 0
    
    if para['w_max'] - (para['k_A']*(R_u + R_o)) <= 0:
        return 0.5
        
    w_D = (para['w_max'] - (para['k_A']*(R_u + R_o)))*prop_d
    utility = para['A']*((w_D)**(para['alpha']))*((R_u)**(para['beta']))

    if returnStab:
        return 1
    else:
        return sign*utility

In [11]:
def behavioral_eqns(rules, para):
    R_u, R_o = rules
    w_A, w_D, w_C, w_P = behavioral_work_eqns((R_u, R_o), para)
    return system_eqns(rules, w_C, w_P, para)

In [22]:
def behavioral_eqns_fsolve(rules, para):
    """
    This function is functionally equivalent to behavioral eqns save that it allows fsolve to evaluate for negative values of rules without an error.
    """
    R_u, R_o = rules
    if R_u < 0 or R_o < 0:
        return [np.inf, np.inf]
    else:
        w_A, w_D, w_C, w_P = behavioral_work_eqns((R_u, R_o), para)
        return system_eqns(rules, w_C, w_P, para)

In [13]:
def x0_func(para):
    w = para['w_max']
    ka = para['k_A']
    init_mesh = 10
    if para['b']  == 0:
        multip = 1
        init_Ro_vals = list(np.linspace(1, 1001, init_mesh)-1)
    elif para['b'] < np.exp(-1):
        multip = -2*np.real(lambertw(-para['b'], -1))/para['b']
        init_Ro_vals = list(np.geomspace(1, multip*para['w_max']/para['k_A'] + 1, init_mesh) - 1)
    else:
        multip = -1.5*np.real(lambertw(np.exp(-0.9999), -1))/np.exp(-0.9999)
        init_Ro_vals = list(np.geomspace(1, multip*para['w_max']/para['k_A'] + 1, init_mesh) - 1)

    init_Ru_vals = list(np.linspace(1, (para['w_max']/para['k_A']), init_mesh))
    init_lists = [init_Ru_vals, init_Ro_vals]
    x0s = [[x,y] for x in init_lists[0] for y in init_lists[1]]
    return x0s

In [14]:
def equilFuncBehavioral(para, all_results = False, x0s = None, xtol = 1e-8):
    """
    Utility function for the stable equilibrium point using the path-independent formulation. 
    sign is a parameter that allows us to change the sign of the output. This allows us to use SciPy's Minimize function.
    """
    # Initial set up
    a = para['a']
    ka = para['k_A']
    w = para['w_max']

    if x0s is None:
        x0s = x0_func(para)
            
    conv_bool = False
    max_U = 0
    max_x0 = [0,0]
    max_result = [0,0]
    results = set()
    
    # Finding R_o
    for x0 in x0s:
        result, info, ier, msg = fsolve(behavioral_eqns_fsolve, x0, args=(para), full_output = True, xtol = xtol)
        # Solving for the utility -- or setting utility to zero if there is no equilibrium value (bureaucratic death)
        if ier == 1 and result[1] >= 0 and result[0] >= 0 and result[0] != x0[0] and result[1] != x0[1]: # Only keep the results from x0 that converge.
            conv_bool = True
            R_u = result[0]
            R_o = result[1]
            w_A, w_D, w_C, w_P = behavioral_work_eqns((R_u, R_o), para)

            utility = para['A']*((w_D)**(para['alpha']))*((R_u)**(para['beta']))
            results.add((round(R_u,2), round(R_o,2)))
            if utility >= max_U:
                max_U = utility
                max_x0 = x0
                max_result = result
    
    if conv_bool:
        if all_results:
            return list(results), 1
        else:
            return max_result, 1, "Converged", max_U
    else:
        if all_results:
            return list(results), 0
        else:
            return max_result, 2, "Did not converge", 0

### Simulate

In [15]:
def simulate(para, generalAlloc = True, tau = 100, T = 100, simulate_equil = False, tol = 1e-6, optimize = False):
    """
    Function to simulate the system over (the next) T timesteps. 
    pair is the tuple (d, c) if decision_strategy is False
         is the triple (dtilde, gamma_c, gamma_p) if decision_strategy is True
    tau is the parameter/values return time (decision period in dynamics optimization)
    T is the cumulative behaviors return time (time horizon in dynamic optimization)
    optimize is a bool determining output structure/times. Optimize returns the cumulative utility times negative one.
    """
    # Unpack/check input values
    if not valid_params(para, generalAlloc, optimize):
        return 0
    if tau > T:
        raise ValueError("Please ensure that tau <= T.")            

    # Initial Rules Values
    R_u = para["init_R_u"]
    R_o = para["init_R_o"]
    R_us = [R_u]
    R_os = [R_o]
    R = R_u+R_o
    R_s = [R]
    old_R = -1
    
    # Initial Work Allocations and Proportions
    if generalAlloc:
        w_A, w_D, w_C, w_P = general_work_eqns((R_u, R_o), para)
    else:
        w_A, w_D, w_C, w_P = behavioral_work_eqns((R_u, R_o), para)
            
     # Initial Utility Counters
    if w_D > 0:
        U = para['A']*((w_D)**(para['alpha']))*((R_u)**(para['beta']))
    else:
        U = 0 
        
    Us = [U]
    disc_Us = [U]
    w_Ds = [w_D]
    w_As = [w_A]
    
    # Initiializing time and continue_condition
    t = 0.
    timesteps = [t]
    continue_condition = True

  # Simulate
    while t < T and continue_condition:
        if simulate_equil:
            if np.abs(R - old_R) > tol:
                continue_condition = True
            else:
                continue_condition = False

      # Times Update
        t = t + para['dt']
        timesteps.append(t)

      # Rule Updates
        dRudt, dRodt = system_eqns((R_u, R_o), w_C, w_P, para)
        R_u = max(R_u + dRudt*para['dt'], 0)
        R_o = max(R_o + dRodt*para['dt'], 0)
        
        R_us.append(R_u)
        R_os.append(R_o)
        old_R = R
        R = R_u+R_o
        R_s.append(R)
        
        
      # Work and Utility Updates
        if generalAlloc:
            w_A, w_D, w_C, w_P = general_work_eqns((R_u, R_o), para)
        else:
            w_A, w_D, w_C, w_P = behavioral_work_eqns((R_u, R_o), para)
    
        U = para['A']*((w_D)**(para['alpha']))*((R_u)**(para['beta']))
        w_Ds.append(w_D)
        w_As.append(w_A)
        Us.append(U)
        disc_Us.append(U/np.exp(t*para['discount_rate']))
   
  # Results
    # Final output data assignments
    if optimize:
        return -1*np.sum(disc_Us)
    elif simulate_equil:
        if continue_condition:
            return [R_us[-1], R_os[-1]], 2, "Ran until max time"
        else:
            return [R_us[-1], R_os[-1]], 1, "Converged early"
    else:
        return R_us, R_os, w_As, w_Ds, Us, disc_Us, timesteps       